question 1

In [1]:
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

x = np.random.rand(20, 7)
y = 3 * x[:, 0] + 2 * x[:, 1] + np.random.randn(20) * 0.1 

# Add bias column (for intercept)
X = np.c_[np.ones(x.shape[0]), x]

scaler = StandardScaler()
x[:, 1:] = scaler.fit_transform(x[:, 1:])

# Ridge Regression using Gradient Descent
def ridge_regression(x, y, lr, lam, epochs=1000):
    m, n = x.shape
    w = np.zeros(n) #model weights
    for _ in range(epochs):
        y_pred = x.dot(w)
        grad = (-2/m) * x.T.dot(y - y_pred) + 2 * lam * w
        grad[0] -= 2 * lam * w[0]   # no regularization for bias term
        w -= lr * grad
        
        if np.any(np.isnan(w)) or np.any(np.isinf(w)):
            return np.zeros(n), np.inf
    cost = np.mean((y - x.dot(w))**2) + lam * np.sum(w**2)
    return w, cost

lrs = [0.0001, 0.001, 0.01, 0.1, 1, 10]
lams = [1e-15, 1e-10, 1e-5, 1e-3, 0, 1, 10, 20]
best = (None, float('inf'), -1)

for lr in lrs:
    for lam in lams:
        w, cost = ridge_regression(x, y, lr, lam)
        r2 = r2_score(y, x.dot(w))
        if cost < best[1] and r2 > best[2]:
            best = (w, cost, r2)

print("Best weights:", best[0])
print("Min Cost:", best[1])
print("Max R2 Score:", best[2])

Best weights: [ 4.66764169  0.6803408  -0.08895294  0.14963576  0.04090136 -0.12017234
 -0.13964323]
Min Cost: 0.21622003273774665
Max R2 Score: 0.8407395414239176


question 4

In [7]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

x, y = load_iris(return_X_y=True)
x = StandardScaler().fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

#logistic regression for one-vs-rest
def train_lr(x, y, lr=0.1, epochs=1000):
    m, n = x.shape
    w = np.zeros(n)
    for _ in range(epochs):
        z = x.dot(w)
        h = sigmoid(z)
        grad = (1/m) * x.T.dot(h - y)
        w -= lr * grad
    return w

weights = []
for cls in np.unique(y):
    y_bin = (y_train == cls).astype(int)
    weights.append(train_lr(x_train, y_bin))
def predict(x):
    probs = [sigmoid(x.dot(w)) for w in weights]
    return np.argmax(probs, axis=0)

y_pred = predict(x_test)
acc = np.mean(y_pred == y_test)
print("Accuracy:", acc)

Accuracy: 0.9
